# CS591 HW2 by Ziran Min

In [399]:
import os
import csv
import subprocess
import re
import random
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Read Files

In [400]:
def read_in_shakespeare():
    tuples = []
    
    with open('will_play_text.csv') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            play_name = row[1]
            line = row[5]
            line_tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', line).split()
            line_tokens = [token.lower() for token in line_tokens]

            tuples.append((play_name, line_tokens))
    
    with open('vocab.txt') as f:
        vocab =  [line.strip() for line in f]
    with open('play_names.txt') as f:
        document_names =  [line.strip() for line in f]
    
    return tuples, document_names, vocab

In [401]:
def get_row_vector(matrix, row_id):
    return matrix[row_id, :]

def get_column_vector(matrix, col_id):
    return matrix[:, col_id]

# Create Matrices

In [402]:
def create_term_document_matrix(line_tuples, document_names, vocab):
    vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
    docname_to_id = dict(zip(document_names, range(0, len(document_names))))
    
    # num of vocabs
    a = len(vocab_to_id)
    # num of documents
    b = len(docname_to_id)
    # initialize result matrix
    result = np.zeros((a,b))
      
    for i in line_tuples: 
        # find each doc index
        doc = docname_to_id[i[0]]
        for j in i[1]:
            # find vocab index 
            word = vocab_to_id[j]
            result[word,doc] +=  1
    
    return result

In [403]:
def create_term_context_matrix(line_tuples, vocab, context_window_size=1):
    vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
    
    # num of vocabs
    n = len(vocab_to_id)
    # initialize result matrix
    result = np.zeros((n,n))

    for i in line_tuples: 
        words_line = i[1]  
        for j, word in enumerate(words_line): 
            # setting upper and lowerbound of range for search words
            upper_bound = min(j + context_window_size + 1, len(words_line))
            lower_bound = max(0, j - context_window_size)
            
            # adding counts into result matrix
            for k in range(lower_bound, upper_bound):
                row_word = vocab_to_id[word]
                col_word = vocab_to_id[words_line[k]] 
                result[row_word,col_word] += 1
  
    return result

# Weighting Terms
### PPMI

In [404]:
def create_PPMI_matrix(term_context_matrix):
    total_count = np.sum(term_context_matrix)
      
    # compute p_ij
    p_ij = (term_context_matrix ) / total_count
    
    # compute p_i
    p_i = np.sum(p_ij, axis=1)[:,None]  
    # compute p_j
    p_j = np.sum(p_ij, axis=0)[None,:]  
  
    temp1 = np.ones(term_context_matrix.shape)
    temp2 = temp1 / p_i
    temp3 = temp2 / p_j
  
    # take log
    result = np.log2(p_ij * temp3)
    # change negative to zero
    result = np.maximum(result, 0)
    
    return result

### tf-idf 

In [405]:
# From Prof. Wijaya on Piazza
# Use the simplest definition of TFIDF
# f = count(t, d) --> term frequency 
# idf = log (N / number_of_docs_with_the_term_t)  --> inverse document frequency, log here is natural log
# tfidf = tf * idf

In [406]:
def create_tf_idf_matrix(term_document_matrix):
    word_num, docs_num = term_document_matrix.shape
    df = np.sum( term_document_matrix > 0, axis=1)
    idf = np.log(docs_num / df)
    tfidf = np.multiply(term_document_matrix, idf.reshape(word_num,1))
    return tfidf

# Similarities 

### Cosine Similarity

In [407]:
def compute_cosine_similarity(vector1, vector2):
    # by formula
    num = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    return num/(norm1*norm2)

### Jaccard Similarity 

In [408]:
def compute_jaccard_similarity(vector1, vector2):
    # by formula
    num = np.sum(np.minimum(vector1,vector2))
    den = np.sum(np.maximum(vector1,vector2))
    return num/den

### Dice Similarity 

In [409]:
def compute_dice_similarity(vector1, vector2):
    # by formula
    num = 2.0 * np.sum(np.minimum(vector1, vector2))
    den = np.sum(vector1+vector2)
    return num/den

# Rank

In [410]:
def rank_plays(target_play_index, term_document_matrix, similarity_fn):
    # num of plays
    n = term_document_matrix.shape[1]
    # initialize simliarities list
    sim_l = np.zeros(n)
    
    # vector of target play
    target_vector = get_column_vector(term_document_matrix, target_play_index)
    
    # find similarities with each other play
    for i in range(n):
        # vector of each other play
        compared_vector = get_column_vector(term_document_matrix, i)
        sim_l[i] = similarity_fn(target_vector, compared_vector)
    
    # sorting and ranking
    result = np.argsort(-sim_l)
    return result

In [411]:
def rank_words(target_word_index, matrix, similarity_fn):
    # num of wrods
    n = matrix.shape[1]
    # initialize simliarities list
    sim_l = np.zeros(n)
    
    # vector of target words
    target_vector = get_row_vector(matrix, target_word_index)
    
    # find similarities with each other words
    for i in range(n):
        # vector of each other words
        compared_vector = get_row_vector(matrix, i)
        sim_l[i] = similarity_fn(target_vector, compared_vector)
    
    # sorting and ranking
    result = np.argsort(-sim_l)
    return result

# Test

In [412]:
print('Computing term document matrix...')
td_matrix = create_term_document_matrix(tuples, document_names, vocab)

Computing term document matrix...


In [413]:
td_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 5., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [414]:
print('Computing tf-idf matrix...')
tf_idf_matrix = create_tf_idf_matrix(td_matrix)

Computing tf-idf matrix...


In [415]:
tf_idf_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 2.19722458, 0.        ,
        0.        ],
       [0.        , 3.4657359 , 0.        , ..., 0.69314718, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.28093385, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [416]:
print('Computing term context matrix...')
tc_matrix = create_term_context_matrix(tuples, vocab, context_window_size=2)

Computing term context matrix...


In [417]:
tc_matrix

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 10.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 47., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., 23.]])

In [418]:
print('Computing PPMI matrix...')
PPMI_matrix = create_PPMI_matrix(tc_matrix)

Computing PPMI matrix...


In [419]:
PPMI_matrix

array([[18.58027329,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , 14.35702238,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 12.28919786, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., 17.1063421 ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        17.1063421 ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 12.7373916 ]])

In [420]:
random_idx = random.randint(0, len(document_names)-1)
similarity_fns = [compute_cosine_similarity, compute_jaccard_similarity, compute_dice_similarity]
for sim_fn in similarity_fns:
    print('\nThe 10 most similar plays to "%s" using %s are:' % (document_names[random_idx], sim_fn.__qualname__))
    ranks = rank_plays(random_idx, td_matrix, sim_fn)
    for idx in range(0, 10):
        doc_id = ranks[idx]
        print('%d: %s' % (idx+1, document_names[doc_id]))


The 10 most similar plays to "King John" using compute_cosine_similarity are:
1: King John
2: Henry VI Part 1
3: Richard II
4: Henry VI Part 2
5: Henry V
6: Richard III
7: Henry IV
8: Hamlet
9: macbeth
10: Cymbeline

The 10 most similar plays to "King John" using compute_jaccard_similarity are:
1: King John
2: Richard II
3: Henry VI Part 1
4: Titus Andronicus
5: Henry IV
6: Henry VI Part 2
7: Henry VI Part 3
8: Pericles
9: Romeo and Juliet
10: Merchant of Venice

The 10 most similar plays to "King John" using compute_dice_similarity are:
1: King John
2: Richard II
3: Henry VI Part 1
4: Titus Andronicus
5: Henry IV
6: Henry VI Part 2
7: Henry VI Part 3
8: Pericles
9: Romeo and Juliet
10: Merchant of Venice


In [421]:
word = 'juliet'
vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on term-document frequency matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], td_matrix, sim_fn)
    for idx in range(0, 10):
        word_id = ranks[idx]
        print('%d: %s' % (idx+1, vocab[word_id]))


The 10 most similar words to "juliet" using compute_cosine_similarity on term-document frequency matrix are:
1: scaring
2: dreamt
3: umpire
4: lovers
5: pipes
6: mood
7: nobleman
8: success
9: fenton
10: couplement

The 10 most similar words to "juliet" using compute_jaccard_similarity on term-document frequency matrix are:
1: lovers
2: dreamt
3: nobleman
4: scaring
5: umpire
6: pipes
7: mood
8: success
9: enticements
10: fenton

The 10 most similar words to "juliet" using compute_dice_similarity on term-document frequency matrix are:
1: lovers
2: dreamt
3: nobleman
4: scaring
5: umpire
6: pipes
7: mood
8: success
9: enticements
10: fenton


In [422]:
word = 'juliet'
vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on term-context frequency matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], tc_matrix, sim_fn)
    for idx in range(0, 10):
        word_id = ranks[idx]
        print('%d: %s' % (idx+1, vocab[word_id]))


The 10 most similar words to "juliet" using compute_cosine_similarity on term-context frequency matrix are:
1: juliet
2: and
3: drinkings
4: pined
5: scamble
6: metheglins
7: groaning
8: cymbals
9: bleated
10: grunt

The 10 most similar words to "juliet" using compute_jaccard_similarity on term-context frequency matrix are:
1: juliet
2: silvia
3: nurse
4: orlando
5: proteus
6: othello
7: demetrius
8: leonato
9: hamlet
10: marcus

The 10 most similar words to "juliet" using compute_dice_similarity on term-context frequency matrix are:
1: juliet
2: silvia
3: nurse
4: orlando
5: proteus
6: othello
7: demetrius
8: leonato
9: hamlet
10: marcus


In [423]:
word = 'juliet'
vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on PPMI matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], tf_idf_matrix, sim_fn)
    for idx in range(0, 10):
        word_id = ranks[idx]
        print('%d: %s' % (idx+1, vocab[word_id]))


The 10 most similar words to "juliet" using compute_cosine_similarity on PPMI matrix are:
1: scaring
2: dreamt
3: umpire
4: lovers
5: pipes
6: mood
7: nobleman
8: success
9: fenton
10: couplement

The 10 most similar words to "juliet" using compute_jaccard_similarity on PPMI matrix are:
1: lovers
2: scaring
3: dreamt
4: umpire
5: nobleman
6: pipes
7: mood
8: success
9: enticements
10: fenton

The 10 most similar words to "juliet" using compute_dice_similarity on PPMI matrix are:
1: lovers
2: scaring
3: dreamt
4: umpire
5: nobleman
6: pipes
7: mood
8: success
9: enticements
10: fenton


In [424]:
word = 'juliet'
vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on PPMI matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], PPMI_matrix, sim_fn)
    for idx in range(0, 10):
        word_id = ranks[idx]
        print('%d: %s' % (idx+1, vocab[word_id]))


The 10 most similar words to "juliet" using compute_cosine_similarity on PPMI matrix are:
1: juliet
2: pined
3: waken
4: capulet
5: tybalt
6: muffled
7: fares
8: provost
9: wills
10: county

The 10 most similar words to "juliet" using compute_jaccard_similarity on PPMI matrix are:
1: juliet
2: tybalt
3: capulet
4: silvia
5: lucio
6: nurse
7: romeo
8: montague
9: leonato
10: provost

The 10 most similar words to "juliet" using compute_dice_similarity on PPMI matrix are:
1: juliet
2: tybalt
3: capulet
4: silvia
5: lucio
6: nurse
7: romeo
8: montague
9: leonato
10: provost
